In [36]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np
from os import listdir
import re

import time

### Retrieve data files list

In [2]:
folder = 'data/'
# get all file names
files = [f for f in listdir(folder) if f[-4:]=='.png']
# load one to get the dimensions
image = mpimg.imread(f'{folder}{files[0]}')
# get dimensions
shape = list(image.shape)
# remove alpha if necessary
if (shape[2]==4):
    shape[2] = 3
n = len(files)

### Read in data

In [3]:
y = np.zeros(shape=n,dtype=int)
X = np.zeros(shape=(n,*shape))
for i in range(len(files)):
    # get the hour from the file name
    y[i] = re.search('^([0-9]*)_',files[i])[1]
    # get the pixels, remove the alpha if needed and convert to 0-255
    X[i] = (mpimg.imread(f'{folder}{files[i]}')[:,:,:shape[2]] )
    
# transform y to label encoded
temp = y.copy()
y = np.zeros(shape=(n,12),dtype=int)
for i in range(n):
    y[i,temp[i]-1] = 1   

### Split data to training and test sets

In [4]:
# note, the input files are already shuffled
train = 0.8
cutoff = int(len(X) * train)

X_train = X[:cutoff]
y_train = y[:cutoff]

X_test = X[cutoff:]
y_test = y[cutoff:]

### Network

In [6]:
import tensorflow as tf

In [8]:
model = Sequential()
model.add(Convolution2D(64, kernel_size=3,activation='relu', input_shape=shape))
model.add(Convolution2D(32, kernel_size=3,activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(12,activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [31]:
def new_weights(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

def new_bias(length):
    return tf.Variable(tf.constant(0.0,shape=[length]))
    
def new_conv_layer(X,channels,k_size,n_kernels,use_pooling=True):
    shape=[k_size,k_size,channels,n_kernels]
    kernels = new_weights(shape)
    bias = new_bias(n_kernels)
    
    layer = tf.nn.conv2d(input=X,filter=kernels,strides=[1,1,1,1],padding='SAME')
    layer+= bias
    
    if use_pooling:
        layer = tf.nn.max_pool(layer,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    
    layer = tf.nn.relu(layer)
    
    return layer, kernels

def flatten(layer):
    shape = layer.shape
    num_features = np.array(shape[1:4],dtype=int).prod()
    layer_flat = tf.reshape(layer,[-1,num_features])
    return layer_flat,num_features

def new_fc_layer(X,num_inputs,num_outputs,use_relu=True):
    weights = new_weights([num_inputs,num_outputs])
    bias = new_bias(num_outputs)
    layer = tf.matmul(X,weights) + bias
    if (use_relu):
        layer = tf.nn.relu(layer)
    return layer

img_size = shape[0]
channels = shape[2]
img_size_flat = np.array(shape).prod()

x = tf.placeholder(tf.float32,shape=[None,img_size_flat],name='x')
x_image = tf.reshape(x,[-1,img_size,img_size,channels])

y_true = tf.placeholder(tf.float32,shape=[None,12],name='y_true')

nk1 = 64
nk2 = 32

layer_conv1, conv1_weights = new_conv_layer(x_image,channels,3,nk1,False)  
layer_conv2, conv2_weights = new_conv_layer(layer_conv1,nk1,3,nk2,True)  
layer_flat,n_n = flatten(layer_conv2)

layer_fc1 = new_fc_layer(layer_flat,n_n,512,True)
layer_fc2 = new_fc_layer(layer_fc1,512,12,False)

y_pred = tf.nn.softmax(layer_fc2)

cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=layer_fc2,labels=y_true)

cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)

pred_labels = tf.math.round(y_pred)
correct_pred = tf.equal(pred_labels,y_true)
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

In [32]:
session = tf.Session()

In [34]:
session.run(tf.global_variables_initializer())

In [35]:
train_batch_size = 25

In [3]:
def optimize(num_iterations,X,y,batch_size):
    batches = int(len(X)/batch_size)
    for j in range(batch):
        x_batch = X[j*batch_size:(j+1)*batch_size]
        y_batch = y[j*batch_size:(j+1)*batch_size]
        feed_dict = {x: x_batch,y_true:y_batch}
        session.run(optimizer,feed_dict)

In [ ]:
session.close()